<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 2 / Assignment 3*

--- 

# Language Modeling with RNN



## Code

다음 링크는 LSTM을 사용하여 Spam 메시지 분류를 수행한 캐글 노트북입니다. => [Link](https://www.kaggle.com/kredy10/simple-lstm-for-text-classification) <br/>

위 노트북에서 사용한 코드를 참고하여<br/>
캐글 데이터셋인 [Women's E-Commerce Clothing Reviews](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews) 를 분류해 보세요.

- 분류에 사용될 텍스트 데이터 : **`Review Text`** 열을 사용합니다.
- 레이블(label) 데이터 : **`Recommended IND`** 열을 사용합니다.

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
#from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
%matplotlib inline

In [3]:
from google.colab import files

file = files.upload()

Saving Womens Clothing E-Commerce Reviews.csv to Womens Clothing E-Commerce Reviews.csv


### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다.
- 필요없는 열(column)을 삭제합니다.

In [4]:
df=pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df=df[['Review Text','Recommended IND']]
df

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",1
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,1


In [5]:
np.random.seed(42)
tf.random.set_seed(42)

In [15]:
#결측치있냐().밸류들중에.어떤거라도()
df.isnull().values.any()

#결측치가있는 행 제거하고 저장
df=df.dropna()
df

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",1
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,1


### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 20%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words=3000` 으로 설정합니다.
- pad_sequence의 `maxlen=400` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=128, epochs=10, validation_split=0.2` 로 설정합니다.
- EarlyStopping을 적용합니다. 파라미터는 `monitor='val_loss',min_delta=0.0001, patience=3` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [26]:
X= df['Review Text']
y= df['Recommended IND']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.15, random_state=42,stratify=y)
tokenizer=Tokenizer(num_words=3000) #빈도가 높은 3000개의 단어 선택 
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)
word_to_index = tokenizer.word_index
max_len = 400
X_train_padded = pad_sequences(X_train_encoded, maxlen = max_len)
embedding_dim = 32
hidden_units = 32

word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=3)
history = model.fit(X_train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2,callbacks=[early_stopping])


Epoch 1/10
121/121 [==============================] - 75s 598ms/step - loss: 0.4306 - acc: 0.8247 - val_loss: 0.3490 - val_acc: 0.8537
Epoch 2/10
121/121 [==============================] - 70s 579ms/step - loss: 0.2923 - acc: 0.8802 - val_loss: 0.3022 - val_acc: 0.8732
Epoch 3/10
121/121 [==============================] - 69s 573ms/step - loss: 0.2372 - acc: 0.9028 - val_loss: 0.3014 - val_acc: 0.8815
Epoch 4/10
121/121 [==============================] - 69s 572ms/step - loss: 0.2061 - acc: 0.9182 - val_loss: 0.2927 - val_acc: 0.8743
Epoch 5/10
121/121 [==============================] - 69s 568ms/step - loss: 0.1764 - acc: 0.9319 - val_loss: 0.2907 - val_acc: 0.8839
Epoch 6/10
121/121 [==============================] - 68s 565ms/step - loss: 0.1398 - acc: 0.9508 - val_loss: 0.3454 - val_acc: 0.8898
Epoch 7/10
121/121 [==============================] - 69s 567ms/step - loss: 0.1108 - acc: 0.9627 - val_loss: 0.3578 - val_acc: 0.8730
Epoch 8/10
121/121 [==============================] - 6